In [1]:
import os
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
import numpy as np

from data.dataset import Experience
from models.vae import Decoder, Encoder, VAE

%load_ext autoreload
%autoreload 2

# Interactive Physics Game
I created a simple interactive physics game using Unity 3D. The game contains a fixed size platform and an object, which the agent can interact with. The object also has a set of latent physics properties (mass, dynamic friction, static friction and bounciness) which are not observed by the agent.

<img src="./physics.png" width="200">
<center><i>Interactive physics game</i></center>

The agent can perform an action and receive an observation at each time step:
- Action: $a = (f_x, f_y)$, a 2D force vector (parallel to the platform) applied to the object.
- Observation: $o = (p_x, p_y, v_x, v_y)$, position and velocity of the object.

The object has a latent physics properties which are not observed by the agent. The physics properties are randomly set at the beginning of each interaction session (episode).

# Latent physics inference
### Notations
- Let $z$ be the latent properties, which is fixed for the object(s) within an episode.
- Let $o_t$ be the observed variable at time $t$
- Let $a_t$ be the action at time $t$
- Interaction history $\xi_t = (o_1, a_1, o_2,...o_{t-1},a_{t-1}, o_t)$

### Belief Update
Suppose the agent holds belief $p(z|\xi_t)$ about $z$ after interaction history $\xi_t$. Then, the agent makes an action $a_t$ and observes $o_{t+1}$. How should the agent update its belief $p(z | \xi_t) \rightarrow p(z | \xi_{t+1})$?

$$
p(z | \xi_{t+1}) \\
= p(z|\xi_t, a_t, o_{t+1}) \\
= p(z | \xi_t)p(o_{t+1}|\xi_t, a_t; z) / p(o_{t+1}|\xi_t, a_t) \\
= p(z|\xi_t) p(o_{t+1}|o_t,a_t;z)/p(o_{t+1}|\xi_t, a_t)
$$

Unfortunately, it is intractable to compute posterior since $p(o_{t+1}|\xi_t, a_t) = \int_z p(o_{t+1}|o_t,a_t;z) dz$. Instead, we can introduce a variational approximation $q(z | \xi_t)$ and minimize ELBO:

$$
\log p(\xi_{t+1}) \geq - \text{KL}[q(z | \xi_{t+1}) || p(z|\xi_t)] + \mathbb{E}_{q(z|\xi_{t+1})} [\log p(o_{t+1} | o_t, a_t; z)]
$$

Here, $q(z | \xi_{t})$ can be implemented as a recurrent interaction encoder, and $p(o_{t+1} | o_t, a_t;z)$ is a forward dynamics model conditioned on latent physics variable $z$.

$q$ and $p$ together compose a recurrent VAE.

### Maximizing information gain
Once we have learned approximate posterior $q$, we can use it to measure the information gain of an one-step interaction:

$$
\text{IG} = H[q(z | \xi_t)] - H[q(z | \xi_t, a_t, o_{t+1})]
$$

where $H[\cdot]$ is the entropy of a probability distribution.

We would like our agent to choose actions that maximally reduce the uncertainty about the latent property $z$. Using the information gain as reward, we can use RL to find a policy that maximizes it.

# Dataset

I collected 20000 episodes of random interactions (100 interactions per episode) using the physics game. The physics properties 

In [2]:
dataset = Experience('data/record', obs_len=50)
data_loader = DataLoader(dataset, batch_size=128,
                         shuffle=True, num_workers=4)

# Training VAE

In [3]:
obs_size = 4
action_size = 2
hidden_size = 32
latent_size = 4

vae = VAE(obs_size, action_size, hidden_size, latent_size)

In [4]:
def KLDivergence(mu1, logsigma1, mu2, logsigma2):
    """ Compute KL(p1 || p2) where p1 ~ N(mu1, sigma1) and p2 ~ N(mu2, sigma2) """
    KLD = - 0.5 * torch.sum(1 + (2 * logsigma1)\
                            - (2 * logsigma2)\
                            - ((2 * logsigma1).exp() + (mu1 - mu2).pow(2)) / (2 * logsigma2).exp())

    return KLD


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logsigma):
    """ VAE loss function """
    B, L, D = mu.size()
    BCE = F.mse_loss(recon_x, x, size_average=False)

    mu0 = torch.zeros((B, 1, D))
    logsigma0 = torch.zeros((B, 1, D))

    mu_prior = torch.cat((mu0, mu), dim=1)[:,:L,:]
    logsigma_prior = torch.cat((logsigma0, logsigma), dim=1)[:,:L,:]

    KLD = KLDivergence(mu, logsigma, mu_prior, logsigma_prior)
    return BCE + KLD

In [5]:
optimizer = optim.Adam(vae.parameters())
vae.train()
for epoch in range(10):
    for batch_idx, data in enumerate(data_loader):
        o, a, o_next = data
        optimizer.zero_grad()
        o_pred, mu, logsigma = vae(o, a, o_next)
        loss = loss_function(o_pred, o_next, mu, logsigma)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(data_loader.dataset),
                    100. * batch_idx / len(data_loader),
                    loss.item() / len(data)))

/home/donlee/Research/WoodLab/env3.6/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 0 [0/20000 (0%)]	Loss: 89981.843750
Train Epoch: 0 [60/20000 (13%)]	Loss: 13730.121094
Train Epoch: 0 [120/20000 (25%)]	Loss: 9610.524089
Train Epoch: 0 [180/20000 (38%)]	Loss: 5700.190755
Train Epoch: 0 [240/20000 (51%)]	Loss: 6374.957031
Train Epoch: 0 [300/20000 (64%)]	Loss: 5629.203125
Train Epoch: 0 [360/20000 (76%)]	Loss: 7026.502604
Train Epoch: 0 [420/20000 (89%)]	Loss: 5298.242839
Train Epoch: 1 [0/20000 (0%)]	Loss: 6255.115234
Train Epoch: 1 [60/20000 (13%)]	Loss: 5762.320312
Train Epoch: 1 [120/20000 (25%)]	Loss: 4582.975260
Train Epoch: 1 [180/20000 (38%)]	Loss: 5321.255534
Train Epoch: 1 [240/20000 (51%)]	Loss: 5902.408203
Train Epoch: 1 [300/20000 (64%)]	Loss: 2452.852539
Train Epoch: 1 [360/20000 (76%)]	Loss: 2515.858887
Train Epoch: 1 [420/20000 (89%)]	Loss: 3015.258464
Train Epoch: 2 [0/20000 (0%)]	Loss: 3360.834310
Train Epoch: 2 [60/20000 (13%)]	Loss: 4067.247721
Train Epoch: 2 [120/20000 (25%)]	Loss: 2734.803711
Train Epoch: 2 [180/20000 (38%)]	Loss: 30

# Sanity check
Let's do a quick sanity check.
First, sample an interaction history and encode it to get a distribution for latent variable $z$.

In [12]:
o, a, o_next = dataset[0]
o = o.unsqueeze(0)
a = a.unsqueeze(0)
o_next = o_next.unsqueeze(0)

In [13]:
mu, logsigma = vae.encoder(o, a, o_next)

Let's examine how the belief changes over a series of interactions. We can see that the mean $\mu$ starts with a value close to zero and converges to some value.

In [17]:
mu[0]

tensor([[-0.2331, -0.2753,  0.3951,  0.2476],
        [-0.3732, -0.4500,  0.6920, -0.0653],
        [-0.3820, -0.5609,  0.9611, -0.1218],
        [-0.3772, -0.7154,  1.1976, -0.1425],
        [-0.4185, -0.8637,  1.3545, -0.1288],
        [-0.4382, -0.9238,  1.4648, -0.1471],
        [-0.5279, -1.1424,  1.5633, -0.2471],
        [-0.5341, -1.1987,  1.6579, -0.2509],
        [-0.5470, -1.2428,  1.7236, -0.2424],
        [-0.5650, -1.3049,  1.7846, -0.2879],
        [-0.5921, -1.3588,  1.8285, -0.2887],
        [-0.6128, -1.3935,  1.8698, -0.2937],
        [-0.6307, -1.4159,  1.9094, -0.3024],
        [-0.6490, -1.4577,  1.9556, -0.3149],
        [-0.6590, -1.4665,  1.9905, -0.3229],
        [-0.6713, -1.4784,  2.0112, -0.3323],
        [-0.6864, -1.5043,  2.0248, -0.3541],
        [-0.7008, -1.5248,  2.0434, -0.3720],
        [-0.7394, -1.5760,  2.0344, -0.4300],
        [-0.7389, -1.5753,  2.0542, -0.4309],
        [-0.7417, -1.5823,  2.0700, -0.4361],
        [-0.7634, -1.6089,  2.0857

The variance $\sigma$ starts from a value close to 1 (similar to prior dist.) and slowly decreases over time. This means that the agent is becoming more confident about its belief, as it gains more observations.

In [18]:
logsigma[0].exp()

tensor([[0.8698, 0.7583, 0.9630, 0.9691],
        [0.7720, 0.6687, 0.9080, 0.8042],
        [0.7423, 0.6072, 0.8732, 0.7616],
        [0.7253, 0.5392, 0.8489, 0.7420],
        [0.6892, 0.4881, 0.8084, 0.7170],
        [0.6586, 0.4565, 0.7772, 0.6817],
        [0.6209, 0.3807, 0.7152, 0.6428],
        [0.5963, 0.3581, 0.6910, 0.6200],
        [0.5780, 0.3403, 0.6682, 0.6024],
        [0.5631, 0.3189, 0.6514, 0.5749],
        [0.5450, 0.3017, 0.6260, 0.5594],
        [0.5293, 0.2870, 0.6036, 0.5454],
        [0.5201, 0.2739, 0.5885, 0.5325],
        [0.5001, 0.2596, 0.5679, 0.5149],
        [0.4941, 0.2467, 0.5581, 0.4995],
        [0.4867, 0.2395, 0.5484, 0.4874],
        [0.4755, 0.2320, 0.5371, 0.4720],
        [0.4700, 0.2252, 0.5285, 0.4628],
        [0.4498, 0.2128, 0.5129, 0.4295],
        [0.4444, 0.2086, 0.5044, 0.4247],
        [0.4403, 0.2051, 0.4979, 0.4200],
        [0.4308, 0.1984, 0.4893, 0.4086],
        [0.4265, 0.1957, 0.4836, 0.4043],
        [0.4181, 0.1905, 0.4746, 0